In [4]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

In [5]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [6]:
connection = create_connection(
    "nicholasmontalbano", "postgres", "", "127.0.0.1", "5432"
)

Connection to PostgreSQL DB successful


In [7]:
curs = connection.cursor()

In [8]:
sql_command = """
SELECT * FROM

(SELECT playerID, 
birthYear, birthMonth, birthDay, nameFirst, nameLast, 
weight, height
FROM people_mlb) a

LEFT JOIN (
SELECT playerID, yearID,
SUM(AB), SUM(R), SUM(H), SUM(DOUBLE), SUM(TRIPLE), SUM(HR), SUM(RBI), SUM(SB), SUM(CS), SUM(BB), 
SUM(SO), SUM(IBB), SUM(HBP), SUM(SH), SUM(SF), SUM(GIDP)
FROM batting_mlb
GROUP BY playerID, yearID) b

USING(playerID)

LEFT JOIN (
SELECT playerID, 
SUM (G_all), SUM(GS), SUM(G_batting), SUM(G_defense), SUM(G_p), SUM(G_c), SUM(G_1b), SUM(G_2b), 
SUM(G_3b), SUM(G_ss), SUM(G_lf), SUM(G_cf), SUM(G_rf), SUM(G_of), SUM(G_dh), SUM(G_ph), SUM(G_pr)
FROM appearances_mlb
GROUP BY playerID) c

USING(playerID)

LEFT JOIN (
SELECT playerID, PP
FROM career_batting_mlb) d

USING(playerID)
"""

df = pd.read_sql(sql_command, connection)
df.head()

,playerid,birthyear,birthmonth,birthday,namefirst,namelast,weight,height,yearid,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,pp
0,aardsda01,1981.0,12.0,27.0,David,Aardsma,215.0,75.0,2004,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,catcher
1,aardsda01,1981.0,12.0,27.0,David,Aardsma,215.0,75.0,2006,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,catcher
2,aardsda01,1981.0,12.0,27.0,David,Aardsma,215.0,75.0,2007,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,catcher
3,aardsda01,1981.0,12.0,27.0,David,Aardsma,215.0,75.0,2008,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,catcher
4,aardsda01,1981.0,12.0,27.0,David,Aardsma,215.0,75.0,2009,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,catcher


In [9]:
df.columns = [df.columns[0], df.columns[1], df.columns[2], df.columns[3], df.columns[4], df.columns[5], df.columns[6], df.columns[7], df.columns[8], 'ab', 'r', 'h', 'double', 'triple', 'hr', 'rbi', 'sb', 'cs', 'bb', 'so', 'ibb', 'hbp', 'sh', 'sf', 'gidp', 'g_all', 'gs', 'g_batting', 'g_defense', 'g_p', 'g_c', 'g_1b', 'g_2b', 'g_3b', 'g_ss', 'g_lf', 'g_cf', 'g_rf', 'g_of', 'g_dh', 'g_ph', 'g_pr', 'pp']
df["avg"] = df["h"] / df["ab"]
df["slg"] = ((df["h"] - df["double"] - df["triple"] - df["hr"]) + (df["double"]*2) + (df["triple"]*3) + (df["hr"]*4)) / df["ab"]

conditions = [
    (df["pp"] == "catcher"), 
    (df["pp"] == "1b"), 
    (df["pp"] == "2b"), 
    (df["pp"] == "3b"),     
    (df["pp"] == "ss"), 
    (df["pp"] == "of"), 
    (df["pp"] == "dh")
]

values = [240, 12, 132, 84, 168, 48, 0]
df["pp_score"] = np.select(conditions, values)
df['yearid'] = pd.to_numeric(df['yearid'])
df['age'] = np.where(df['birthmonth']>7, df['yearid'] - df['birthyear'] - 1, df['yearid'] - df['birthyear'])

# find NAs and replace with 0 (could be none or NaN)
df.isnull().values.any()
df.isnull().sum()
df = df.fillna(0)

df = df[df['yearid'] != 0]
career = df.iloc[:,np.r_[0,7:40]].groupby(['playerid']).sum()
five_playerid = career[career['ab'] >= 500].index
boolean_series = df.playerid.isin(five_playerid)
filtered_df = df[boolean_series]
filtered_df.head()

,playerid,birthyear,birthmonth,birthday,namefirst,namelast,weight,height,yearid,ab,...,g_rf,g_of,g_dh,g_ph,g_pr,pp,avg,slg,pp_score,age
9,aaronha01,1934.0,2.0,5.0,Hank,Aaron,180.0,72.0,1954.0,468.0,...,2174.0,2760.0,201.0,122.0,1.0,of,0.279915,0.446581,48,20.0
10,aaronha01,1934.0,2.0,5.0,Hank,Aaron,180.0,72.0,1955.0,602.0,...,2174.0,2760.0,201.0,122.0,1.0,of,0.313953,0.539867,48,21.0
11,aaronha01,1934.0,2.0,5.0,Hank,Aaron,180.0,72.0,1956.0,609.0,...,2174.0,2760.0,201.0,122.0,1.0,of,0.328407,0.558292,48,22.0
12,aaronha01,1934.0,2.0,5.0,Hank,Aaron,180.0,72.0,1957.0,615.0,...,2174.0,2760.0,201.0,122.0,1.0,of,0.321951,0.600000,48,23.0
13,aaronha01,1934.0,2.0,5.0,Hank,Aaron,180.0,72.0,1958.0,601.0,...,2174.0,2760.0,201.0,122.0,1.0,of,0.326123,0.545757,48,24.0


In [10]:
player = 'lindofr01'
sub = filtered_df[filtered_df['age'] <= filtered_df.loc[filtered_df['playerid'] == player, 'age'].nlargest(1).item()]
df_age = sub.iloc[:,np.r_[0,9:43]].groupby(['playerid']).sum()
df_age = df_age.merge(df[['playerid', 'weight', 'height', 'pp', 'pp_score']].drop_duplicates(), on='playerid', how='left')
df_age.columns

Index(['playerid', 'ab', 'r', 'h', 'double', 'triple', 'hr', 'rbi', 'sb', 'cs',
       'bb', 'so', 'ibb', 'hbp', 'sh', 'sf', 'gidp', 'g_all', 'gs',
       'g_batting', 'g_defense', 'g_p', 'g_c', 'g_1b', 'g_2b', 'g_3b', 'g_ss',
       'g_lf', 'g_cf', 'g_rf', 'g_of', 'g_dh', 'g_ph', 'g_pr', 'weight',
       'height', 'pp', 'pp_score'],
      dtype='object')

In [11]:
df_age.loc[df_age['playerid']=='lindofr01']

,playerid,ab,r,h,double,triple,hr,rbi,sb,cs,...,g_cf,g_rf,g_of,g_dh,g_ph,g_pr,weight,height,pp,pp_score
2630,lindofr01,3140.0,508.0,896.0,191.0,15.0,138.0,411.0,99.0,27.0,...,0.0,0.0,0.0,66.0,36.0,0.0,190.0,71.0,ss,168


In [48]:
nump_df = df_age[df_age.columns.difference(['playerid', 'g_all', 'gs', 'g_batting', 'g_defense', 'g_p', 'g_c','g_1b', 'g_2b', 'g_3b', 'g_ss','g_lf', 'g_cf', 'g_rf', 'g_of', 'g_dh', 'g_ph', 'g_pr', 'pp'])].to_numpy()
nump_df_normalized = normalize(nump_df, axis=0)
nbrs = NearestNeighbors(n_neighbors=9, algorithm='ball_tree')
nbrs.fit(nump_df_normalized)
distances, indices = nbrs.kneighbors(nump_df_normalized)

In [49]:
# Lindor Index
indices[2630]

array([2630, 3645,  342, 4887, 4534, 4068,  714,  562, 3646])

In [50]:
distances

array([[0.        , 0.05312025, 0.05538635, ..., 0.06280729, 0.06580918,
        0.06860786],
       [0.        , 0.00708125, 0.00726876, ..., 0.00768537, 0.00770115,
        0.00776904],
       [0.        , 0.00499194, 0.00594622, ..., 0.00700632, 0.00702661,
        0.00719473],
       ...,
       [0.        , 0.02261809, 0.02716225, ..., 0.03156987, 0.03253779,
        0.0336983 ],
       [0.        , 0.00909457, 0.00930548, ..., 0.01039229, 0.01051656,
        0.01054149],
       [0.        , 0.01317082, 0.01524177, ..., 0.01636455, 0.01646347,
        0.01648647]])

In [51]:
df_age.loc[indices[2630],]

,playerid,ab,r,h,double,triple,hr,rbi,sb,cs,...,g_cf,g_rf,g_of,g_dh,g_ph,g_pr,weight,height,pp,pp_score
2630,lindofr01,3140.0,508.0,896.0,191.0,15.0,138.0,411.0,99.0,27.0,...,0.0,0.0,0.0,66.0,36.0,0.0,190.0,71.0,ss,168
3645,ramirjo01,2774.0,451.0,776.0,197.0,22.0,110.0,391.0,117.0,28.0,...,0.0,0.0,350.0,84.0,98.0,91.0,190.0,69.0,3b,84
342,bettsmo01,3203.0,613.0,965.0,229.0,26.0,139.0,470.0,126.0,25.0,...,1158.0,3822.0,4902.0,78.0,36.0,24.0,180.0,69.0,of,48
4887,wrighda03,3185.0,552.0,983.0,222.0,13.0,140.0,561.0,119.0,31.0,...,0.0,0.0,0.0,18.0,60.0,6.0,205.0,72.0,3b,84
4534,tulowtr01,2632.0,438.0,770.0,152.0,21.0,122.0,443.0,51.0,28.0,...,0.0,0.0,0.0,24.0,150.0,0.0,205.0,75.0,ss,168
4068,sheffga01,2696.0,399.0,774.0,139.0,12.0,117.0,430.0,96.0,43.0,...,0.0,9280.0,13128.0,2416.0,528.0,16.0,190.0,71.0,of,48
714,cartega01,2950.0,406.0,787.0,135.0,16.0,126.0,444.0,28.0,19.0,...,0.0,924.0,959.0,0.0,917.0,7.0,205.0,74.0,catcher,240
562,brunato01,3297.0,452.0,825.0,153.0,13.0,165.0,469.0,36.0,30.0,...,567.0,10983.0,11753.0,427.0,441.0,7.0,210.0,76.0,of,48
3646,ramirma02,2509.0,442.0,758.0,169.0,6.0,154.0,517.0,25.0,19.0,...,0.0,5424.0,11628.0,1992.0,330.0,30.0,225.0,72.0,of,48


In [52]:
playerid = tuple(df_age.loc[indices[2630], 'playerid'].to_list())
playerid

('lindofr01',
 'ramirjo01',
 'bettsmo01',
 'wrighda03',
 'tulowtr01',
 'sheffga01',
 'cartega01',
 'brunato01',
 'ramirma02')

In [53]:
sql_command = """
SELECT * FROM

(SELECT * 
FROM batting_mlb
WHERE playerid IN {}) a

LEFT JOIN (
SELECT playerID, birthYear, birthMonth, birthDay, nameFirst, nameLast, weight, height
FROM people_mlb) b

USING(playerID)
""".format(playerid)

comp = pd.read_sql(sql_command, connection)

In [54]:
comp = comp.fillna(0)
comp["avg"] = comp["h"] / comp["ab"]
comp["slg"] = (comp["h"] + comp["double"] + (comp["triple"]*2) + (comp["hr"]*3)) / comp["ab"]
comp['obp'] = (comp['h'] + comp['bb'] + comp['hbp']) / (comp['ab'] + comp['bb'] + comp['hbp'] + comp['sf']) # Walks include IBB
comp['ops'] = comp['obp'] + comp['slg']
comp['yearid'] = pd.to_numeric(comp['yearid'])
comp['age'] = np.where(comp['birthmonth']>7, comp['yearid'] - comp['birthyear'] - 1, comp['yearid'] - comp['birthyear'])
comp.to_csv(r'data/tables/knn_lindor_2021.csv', index=False)
comp

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,double,...,birthday,namefirst,namelast,weight,height,avg,slg,obp,ops,age
0,cartega01,1974,1,MON,NL,9,27,5,11,0,...,8,Gary,Carter,205,74,0.407407,0.592593,0.413793,1.006386,20
1,cartega01,1975,1,MON,NL,144,503,58,136,20,...,8,Gary,Carter,205,74,0.270378,0.415507,0.360345,0.775852,21
2,cartega01,1978,1,MON,NL,157,533,76,136,27,...,8,Gary,Carter,205,74,0.255159,0.422139,0.335537,0.757676,24
3,cartega01,1976,1,MON,NL,91,311,31,68,8,...,8,Gary,Carter,205,74,0.218650,0.308682,0.286957,0.595638,22
4,cartega01,1977,1,MON,NL,154,522,86,148,29,...,8,Gary,Carter,205,74,0.283525,0.524904,0.356419,0.881323,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,ramirjo01,2019,1,CLE,AL,129,482,68,123,33,...,17,Jose,Ramirez,190,69,0.255187,0.479253,0.326568,0.805821,26
126,tulowtr01,2019,1,NYA,AL,5,11,1,2,1,...,10,Troy,Tulowitzki,205,75,0.181818,0.545455,0.307692,0.853147,34
127,bettsmo01,2020,1,LAN,NL,55,219,47,64,9,...,7,Mookie,Betts,180,69,0.292237,0.561644,0.365854,0.927497,27
128,ramirjo01,2020,1,CLE,AL,58,219,45,64,16,...,17,Jose,Ramirez,190,69,0.292237,0.607306,0.385827,0.993133,27
